In [1]:
%matplotlib inline

import pandas as pd
import cv2
from matplotlib import pyplot as plt
import torch
from PIL import Image
import numpy as np
import requests
import os

torch.cuda.is_available()
torch.cuda.device_count()

1

In [2]:
test_df = pd.read_csv('../../data/Car Detection/Car Links/Data.csv')

In [3]:
test_df.head()

,Path,XMin,YMin,XMax,YMax,ImageID,Url
0,1.png,0.000000,0.788333,0.999375,0.999167,00005903203677bc,https://c5.staticflickr.com/6/5590/14864523078...
1,2.png,0.023496,0.888750,0.169173,0.965625,00041da5b52f4197,https://farm8.staticflickr.com/3017/5826289740...
2,3.png,0.170625,0.517137,0.323750,0.833669,000355964e21d5c7,https://farm8.staticflickr.com/5331/6916642150...
3,4.png,0.030000,0.000000,0.643125,0.727273,0001cd76a5b18775,https://farm6.staticflickr.com/5134/5420548306...
4,5.png,0.720625,0.584934,0.808125,0.807976,000692d348b0f181,https://farm2.staticflickr.com/7453/1657261338...


In [4]:
test_df['file_name'] = test_df['Url'].str.split('/').str[-1]

In [5]:
test_df.head()

,Path,XMin,YMin,XMax,YMax,ImageID,Url,file_name
0,1.png,0.000000,0.788333,0.999375,0.999167,00005903203677bc,https://c5.staticflickr.com/6/5590/14864523078...,14864523078_90da1581b4_o.jpg
1,2.png,0.023496,0.888750,0.169173,0.965625,00041da5b52f4197,https://farm8.staticflickr.com/3017/5826289740...,5826289740_ab72bf8475_o.jpg
2,3.png,0.170625,0.517137,0.323750,0.833669,000355964e21d5c7,https://farm8.staticflickr.com/5331/6916642150...,6916642150_6845e174f5_o.jpg
3,4.png,0.030000,0.000000,0.643125,0.727273,0001cd76a5b18775,https://farm6.staticflickr.com/5134/5420548306...,5420548306_9345c7967b_o.jpg
4,5.png,0.720625,0.584934,0.808125,0.807976,000692d348b0f181,https://farm2.staticflickr.com/7453/1657261338...,16572613382_10c1de5fb1_o.jpg


In [6]:
test_df.to_csv('../../data/Car Detection/Car Links/Data_fixed.csv')

In [7]:
def download_images():
    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:71.0) Gecko/20100101 Firefox/71.0'}

    parent_folder = '../../data/Car Detection/Car Links/downloaded_images'
    for index, row in test_df.iterrows():
        print(row.Url)
        img_data = requests.get(row.Url, headers=headers)
        if img_data.status_code != 200:
            continue

        file_name = row.Url.split('/')[-1]
        save_path = os.path.join(parent_folder,file_name)

        with open(save_path, 'wb') as handler:
            handler.write(img_data.content)

In [8]:
image_row = test_df[test_df['file_name'] == '10160051035_ee82a8c255_o.jpg']
image_row

,Path,XMin,YMin,XMax,YMax,ImageID,Url,file_name
158,159.png,0.0,0.399625,0.221875,0.999062,0000d6d1c44decb6,https://farm7.staticflickr.com/3831/1016005103...,10160051035_ee82a8c255_o.jpg


In [9]:
image_row.iloc[0].XMax

np.float64(0.221875)

In [10]:
(float(image_row.XMin), float(image_row.YMin))

C:\Users\joshu_rdnqgbx\AppData\Local\Temp\ipykernel_6480\1376914942.py:1: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  (float(image_row.XMin), float(image_row.YMin))


(0.0, 0.399625)

In [19]:
file = cv2.imread('../../data/Car Detection/Car Links/downloaded_images/35069779_23b36594e5_o.jpg')
# cv2.imshow("Image", file)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [20]:
file.shape

(3456, 2304, 3)

In [12]:
(float(image_row.XMin), float(image_row.YMin))

C:\Users\joshu_rdnqgbx\AppData\Local\Temp\ipykernel_6480\1376914942.py:1: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  (float(image_row.XMin), float(image_row.YMin))


(0.0, 0.399625)

In [13]:
(float(image_row.XMax), float(image_row.YMax))

C:\Users\joshu_rdnqgbx\AppData\Local\Temp\ipykernel_6480\3326821292.py:1: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  (float(image_row.XMax), float(image_row.YMax))


(0.221875, 0.999062)

In [24]:
def get_box_positions(image_width,image_height,xmin,ymin,xmax,ymax):
    xmin_val = int(image_width*xmin)
    ymin_val = int(image_height*ymin)
    xmax_val = int(image_width*xmax)
    ymax_val = int(image_height*ymax)

    return (xmin_val,ymin_val),(xmax_val,ymax_val)

In [25]:
parent_folder = '../../data/Car Detection/Car Links/downloaded_images'
for file in os.listdir(parent_folder):
    full_file_path = os.path.join(parent_folder,file)
    image = cv2.imread(full_file_path)
    image_row = test_df[test_df['file_name'] == file].iloc[0]

    box_pos = get_box_positions(image.shape[1],image.shape[0],float(image_row.XMin),float(image_row.YMin),float(image_row.XMax),float(image_row.YMax))
    print(box_pos)
    rec_image = cv2.rectangle(image, box_pos[0],box_pos[1], (255, 0, 0), 2)
    cv2.namedWindow('Image', cv2.WINDOW_NORMAL)
    cv2.imshow('Image', rec_image) 
    cv2.waitKey(0)
    cv2.destroyAllWindows()

((0, 1457), (1214, 3644))
((8, 95), (124, 376))
((0, 584), (503, 1259))
((853, 26), (1100, 282))
((0, 302), (133, 386))
((0, 0), (505, 1305))
((0, 923), (275, 2122))
((0, 282), (5180, 3443))
((551, 1775), (946, 1966))


KeyboardInterrupt: 

In [ ]:
image.shape

In [16]:
start_point = (100, 100)
end_point = (400, 300)
rec_image = cv2.rectangle(file, start_point, end_point, (255, 0, 0), 2)
cv2.imshow('Image', rec_image) 
cv2.waitKey(0)
cv2.destroyAllWindows()

error: OpenCV(4.11.0) :-1: error: (-5:Bad argument) in function 'rectangle'
> Overload resolution failed:
>  - img is not a numpy array, neither a scalar
>  - img is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'img'
>  - Expected Ptr<cv::UMat> for argument 'img'
